#Building a song recommender


#Fire up GraphLab Create

In [5]:
import turicreate as tc

#Load music data

In [6]:
song_data = tc.SFrame('datasets/song_data.gl/')

#Explore data

Music data shows how many times a user listened to a song, as well as the details of the song.

In [7]:
song_data.head()

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


##Showing the most popular songs in the dataset

In [8]:
song_data['song'].show() #aqui vemos en histograma agrupado por cancion!

Materializing SArray...

Done.

Materializing SArray...

Done.

In [9]:
len(song_data) ##el numero de canciones distintas

1116609

##Count number of unique users in the dataset

In [10]:
users = song_data['user_id'].unique() #dame los usuaios distintos
#claro la desagregacion minima es cancion

In [11]:
len(users)

66346

##Contamos para artistas en particular

In [12]:
kanye=song_data[song_data['artist']=='Kanye West']
u_kanye=kanye['user_id'].unique()
len(u_kanye)

2522

In [13]:
foo=song_data[song_data['artist']=='Foo Fighters']
u_foo=foo['user_id'].unique()
len(u_foo)

2055

In [14]:
taylor=song_data[song_data['artist']=='Taylor Swift']
u_taylor=taylor['user_id'].unique()
len(u_taylor)

3246

In [15]:
gaga=song_data[song_data['artist']=='Lady GaGa']
u_gaga=gaga['user_id'].unique()
len(u_gaga)

2928

##Vemos la popularidad de los artistas

In [17]:
listen_count=song_data.groupby(key_column_names='artist', operations={'total_count': tc.aggregate.SUM('listen_count')})
listen_count=listen_count.sort('total_count',ascending=False)
listen_count

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


#Create a song recommender

In [18]:
train_data,test_data = song_data.random_split(.8,seed=0)

##Simple popularity-based recommender

In [32]:
popularity_model = tc.popularity_recommender.create(train_data,
                                                         user_id='user_id',
                                                         item_id='song',
                                                        target='listen_count')
#el mas basico, te muestra lo mas escuchado
#el item que se va a recomendar es la cancion

Recsys training: model = popularity

Warning: Ignoring columns song_id, title, artist;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

Data prepared in: 1.06615s

893580 observations to process; with 9952 unique items.

###Use the popularity model to make some predictions

A popularity model makes the same prediction for all users, so provides no personalization.

In [33]:
popularity_model.recommend(users=[users[0]]) 
#recomendacion para un usuario en particular

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Crack Under Pressure -Righteous Pigs ...,24.1739130435,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,221 - keller williams,19.0777777778,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Hounds of Love (new mix)- The Futureheads ...,15.5,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Working With Homesick -Homesick James ...,14.8441558442,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,The Good Fight -Dashboard Confessional ...,14.5,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Aguas De Marco - JoãoGilberto ...,14.488372093,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Clara meets Slope - HardTo Say - Clara Hill ...,14.4863387978,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cold Blooded (AcidCleanse) - The fFormula ...,13.7352941176,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Caroline - CREMATORY,13.2666666667,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Kiss Me Again Stranger -Doris Day ...,12.8823529412,10


In [34]:
popularity_model.recommend(users=[users[1]])

user_id,song,score,rank
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Crack Under Pressure -Righteous Pigs ...,24.1739130435,1
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,221 - keller williams,19.0777777778,2
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Hounds of Love (new mix)- The Futureheads ...,15.5,3
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Working With Homesick -Homesick James ...,14.8441558442,4
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,The Good Fight -Dashboard Confessional ...,14.5,5
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Aguas De Marco - JoãoGilberto ...,14.488372093,6
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Clara meets Slope - HardTo Say - Clara Hill ...,14.4863387978,7
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Cold Blooded (AcidCleanse) - The fFormula ...,13.7352941176,8
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Caroline - CREMATORY,13.2666666667,9
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Kiss Me Again Stranger -Doris Day ...,12.8823529412,10


##Build a song recommender with personalization

We now create a model that allows us to make personalized recommendations to each user. 

In [36]:
personalized_model = tc.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song',
                                                                target='listen_count')
#modelo de items similares
#lo mismo pero otra funcion

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, title, artist;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 1.90s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.03313s

In [37]:
#hacemos un subset de los usuarios
subset_test_users = test_data['user_id'].unique()[0:10000]

In [38]:
#recomendacion para el subset de usuarios
model_person=personalized_model.recommend(subset_test_users,k=1)
model_person.sort('score',ascending=False)

recommendations finished on 1000/10000 queries. users per second: 3017.57

recommendations finished on 2000/10000 queries. users per second: 2951.21

recommendations finished on 3000/10000 queries. users per second: 3052.04

recommendations finished on 4000/10000 queries. users per second: 3100.71

recommendations finished on 5000/10000 queries. users per second: 3131.05

...

recommendations finished on 7000/10000 queries. users per second: 3092.3

recommendations finished on 8000/10000 queries. users per second: 3132.59

recommendations finished on 9000/10000 queries. users per second: 3142.43

recommendations finished on 10000/10000 queries. users per second: 3122.21

user_id,song,score,rank
dfaeace8a6b4ef0eba8fd415f9b23a3937bd60c4 ...,Auburn and Ivory - BeachHouse ...,0.418406069279,1
10a423e24e4e27587872e9994704d1af5c4d0532 ...,Precious - EsperanzaSpalding / Leo Genovese ...,0.393863305449,1
4f0bf63719f31bba812145be44c209b76a07c378 ...,Bella - Angus & JuliaStone ...,0.379746854305,1
55d80dd7a7619e180906a09920eb705c7c9a362f ...,Una Música Brutal -Gotan Project ...,0.328649759293,1
05dbf0642d324eff161e7b31b51bb9e6f4fd24d2 ...,Fly - Ludovico Einaudi,0.323943674564,1
b5d3ea69a3723c168716c37544c978549242cf15 ...,Bottom Of the Barrel -Amos Lee ...,0.308823525906,1
e065bca16142fe9ef8768cd41342333201496827 ...,On The Road Again - KatieMelua ...,0.305684059858,1
051283b7eb91b7f51f904214ef0a2a038b4e68fd ...,Amor de Loca Juventud -Buena Vista Social Club ...,0.303066631158,1
c4867d2d9356166dc887da97a3111dfa5e08e1f2 ...,Every Time You Go - EllieGoulding ...,0.299260504544,1
4b2b3e036c1a8578ccc8de57cabd29ee0546fc29 ...,The Go Getter - The BlackKeys ...,0.297881181041,1


In [39]:
model_count=model_person.groupby(key_column_names='song',operations={'count': tc.aggregate.COUNT('user_id')})
model_count=model_count.sort('count',ascending=False)
model_count

song,count
Undo - Björk,421
Secrets - OneRepublic,410
Revelry - Kings Of Leon,230
You're The One - DwightYoakam ...,154
Sehr kosmisch - Harmonia,121
Hey_ Soul Sister - Train,107
Horn Concerto No. 4 in Eflat K495: II. Romance ...,89
Fireflies - CharttraxxKaraoke ...,88
OMG - Usher featuringwill.i.am ...,75
Dog Days Are Over (RadioEdit) - Florence + The ...,51


###Applying the personalized model to make song recommendations

As you can see, different users get different recommendations now.

In [40]:
personalized_model.recommend(users=[users[0]])
#modelo. funcion recomendacion (usuario en particular)
#vemoa ahora diferencia

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.0194504536115,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,0.0144737317012,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Love Is A Losing Game -Amy Winehouse ...,0.0142865960415,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Marry Me - Train,0.014133471709,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,0.013591665488,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,0.0133987894425,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Te Hacen Falta Vitaminas- Soda Stereo ...,0.0129302831796,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,OMG - Usher featuringwill.i.am ...,0.0127778282532,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Y solo se me ocurreamarte (Unplugged) - ...,0.0123411279458,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,No Dejes Que... -Caifanes ...,0.0121042499175,10


In [41]:
personalized_model.recommend(users=[users[1]])

user_id,song,score,rank
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Where The Boat LeavesFrom (Album) - Zac Brown ...,0.0615360885859,1
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Different Kind Of Fine(Album) - Zac Brown Band ...,0.0605283752084,2
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Jolene (Album) - ZacBrown Band ...,0.0578682050109,3
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Sic 'Em On A Chicken(Album) - Zac Brown Band ...,0.0551866963506,4
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Who's Kissing You Tonight- Jason Aldean ...,0.0530633330345,5
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,What Country Is - LukeBryan ...,0.0374908074737,6
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Highway 20 Ride (Album) -Zac Brown Band ...,0.0373315736651,7
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Do I - Luke Bryan,0.0330773591995,8
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,One Fine Wire - ColbieCaillat ...,0.03125,9
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Midnight Bottle - ColbieCaillat ...,0.0307377055287,10


###We can also apply the model to find similar songs to any song in the dataset

In [42]:
personalized_model.get_similar_items(['With Or Without You - U2'])
#ahora recomendamos canciones similares a las que ya me gustan
#se debe poner el nombre expreso del item

song,similar,score,rank
With Or Without You - U2,I Still Haven't FoundWhat I'm Looking For ...,0.042857170105,1
With Or Without You - U2,Hold Me_ Thrill Me_ KissMe_ Kill Me - U2 ...,0.0337349176407,2
With Or Without You - U2,Window In The Skies - U2,0.0328358411789,3
With Or Without You - U2,Vertigo - U2,0.0300751924515,4
With Or Without You - U2,Sunday Bloody Sunday - U2,0.0271317958832,5
With Or Without You - U2,Bad - U2,0.0251798629761,6
With Or Without You - U2,A Day Without Me - U2,0.0237154364586,7
With Or Without You - U2,Another Time AnotherPlace - U2 ...,0.0203251838684,8
With Or Without You - U2,Walk On - U2,0.0202020406723,9
With Or Without You - U2,Get On Your Boots - U2,0.0196850299835,10


In [43]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

song,similar,score,rank
Chan Chan (Live) - BuenaVista Social Club ...,Murmullo - Buena VistaSocial Club ...,0.188118815422,1
Chan Chan (Live) - BuenaVista Social Club ...,La Bayamesa - Buena VistaSocial Club ...,0.18719214201,2
Chan Chan (Live) - BuenaVista Social Club ...,Amor de Loca Juventud -Buena Vista Social Club ...,0.184834122658,3
Chan Chan (Live) - BuenaVista Social Club ...,Diferente - Gotan Project,0.0214592218399,4
Chan Chan (Live) - BuenaVista Social Club ...,Mistica - Orishas,0.0205761194229,5
Chan Chan (Live) - BuenaVista Social Club ...,Hotel California - GipsyKings ...,0.0193049907684,6
Chan Chan (Live) - BuenaVista Social Club ...,Nací Orishas - Orishas,0.0191571116447,7
Chan Chan (Live) - BuenaVista Social Club ...,Gitana - Willie Colon,0.018796980381,8
Chan Chan (Live) - BuenaVista Social Club ...,Le Moulin - Yann Tiersen,0.018796980381,9
Chan Chan (Live) - BuenaVista Social Club ...,Criminal - Gotan Project,0.0187793374062,10


#Quantitative comparison between the models

We now formally compare the popularity and the personalized models using precision-recall curves. 

In [44]:
%matplotlib inline
model_performance = tc.recommender.util.compare_models(test_data,
                                                            [popularity_model,personalized_model],
                                                            user_sample=0.05)
#presicion-recall curves nos van a ayudar 
#literal es la funcion compare
#agarra el test data (siempre este para evaluar)
#entre corchetes los dos modelos
#el user sample es tomado del test data

compare_models: using 2931 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/2931 queries. users per second: 2939.86

recommendations finished on 2000/2931 queries. users per second: 2887.04


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    | 0.000341180484476 | 6.82360968953e-05 |
|   2    | 0.000341180484476 | 0.000116976166106 |
|   3    | 0.000227453656318 | 0.000116976166106 |
|   4    | 0.000170590242238 | 0.000116976166106 |
|   5    | 0.000136472193791 | 0.000116976166106 |
|   6    | 0.000113726828159 | 0.000116976166106 |
|   7    | 0.000146220207633 | 0.000230702994265 |
|   8    | 0.000213237802798 | 0.000347679160371 |
|   9    | 0.000189544713598 | 0.000347679160371 |
|   10   | 0.000170590242238 | 0.000347679160371 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]


Overall RMSE: 6.17468945731

Per User RMSE (best)
+-------------------------------+-------+------+
|            user_id            | count | rmse |
+-------------------------------+-------+------+
| 20fc199c2165

recommendations finished on 1000/2931 queries. users per second: 2648.56

recommendations finished on 2000/2931 queries. users per second: 2684.67


Precision and recall summary statistics by cutoff
+--------+-----------------+------------------+
| cutoff |  mean_precision |   mean_recall    |
+--------+-----------------+------------------+
|   1    |  0.029341521665 | 0.00750023508468 |
|   2    | 0.0354827703855 | 0.0190259014082  |
|   3    | 0.0369612191516 | 0.0294526405964  |
|   4    | 0.0356533606278 |  0.037843973476  |
|   5    | 0.0339815762538 | 0.0444632924034  |
|   6    |  0.032980780166 | 0.0509750077816  |
|   7    |  0.033045766925 | 0.0589631396694  |
|   8    |  0.032923916752 | 0.0660940863602  |
|   9    | 0.0327912354524 | 0.0736402860379  |
|   10   | 0.0334698055271 | 0.0854125649929  |
+--------+-----------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 6.99112160357

Per User RMSE (best)
+-------------------------------+-------+----------------+
|            user_id            | count |      rmse      |
+-------------------------------+-------+----------------+
| 7c5a890a73d609f570ca5cac

The curve shows that the personalized model provides much better performance. 

In [45]:
#mientras mas grande mejor
#recordar que es como calcular el area bajo la curva